In [1]:
#https://keras.io/preprocessing/image/
#

In [34]:
HOMEPATH = "/home/ubuntu/fastai/"
epochs = 5
batch_size = 64

In [46]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras import utils as np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from sklearn.preprocessing import OneHotEncoder

import os
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())
#set cwd to load utilities
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

('current working directory:', '/home/ubuntu/fastai')


In [37]:
print keras.__version__

1.2.2


In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [23]:
print ("x_train:", type(x_train), x_train.shape)
print ("y_train:", type(y_train), y_train.shape, np.unique(y_train))
print ("x_test:", type(x_test), x_test.shape)
print ("y_test:", type(y_test), y_test.shape, np.unique(y_test))

('x_train:', <type 'numpy.ndarray'>, (50000, 32, 32, 3))
('y_train:', <type 'numpy.ndarray'>, (50000, 1), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8))
('x_test:', <type 'numpy.ndarray'>, (10000, 32, 32, 3))
('y_test:', <type 'numpy.ndarray'>, (10000, 1), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))


In [42]:
print np.unique(x_train)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [43]:
print np.unique(x_test)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [44]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [21]:
#this doesn't work - probe keras 2.x?
#y_train = np_utils.to_categorical(y_train, num_classes)
#y_test = np_utils.to_categorical(y_test, num_classes)

#use onhotEncoder instead.
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [22]:
y_train_hot = onehot(y_train)
print(y_train_hot.shape)
y_test_hot = onehot(y_test)
print(y_test_hot.shape)

(50000, 10)
(10000, 10)


In [19]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [24]:
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

In [53]:
x_train.shape[1:]

(32, 32, 3)

In [52]:
model = Sequential()
model.add(Conv2D(32, (3, 3), border='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



TypeError: __init__() takes at least 4 arguments (3 given)

In [41]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                    samples_per_epoch=len(x_train) / batch_size, nb_epoch=epochs)

Epoch 1/5


ValueError: Error when checking model input: expected lambda_input_1 to have shape (None, 3, 224, 224) but got array with shape (64, 32, 32, 3)